In [1]:
import json
import random
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_groq import ChatGroq
from langchain_core.output_parsers import StrOutputParser, PydanticOutputParser
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, AIMessage
from langchain_community.tools.tavily_search import TavilySearchResults

from langgraph.graph import StateGraph, MessagesState, START, END, MessageGraph
from IPython.display import display, Image
from typing import TypedDict, Annotated, Literal
from pydantic import BaseModel, Field
import operator
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import ToolNode

load_dotenv("../.env")
llm = ChatGroq(
    # model="llama3-groq-70b-8192-tool-use-preview",
    model="llama-3.1-70b-versatile",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)
#embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

memory = MemorySaver()
config = {"configurable": {"thread_id": 1}}

In [2]:
# Import tools from tools.py
from tools import get_current_weather, get_system_time

In [3]:
model_with_tools = llm.bind_tools(
    tools=[get_current_weather, get_system_time],
)

tool_mapping = {
    'get_current_weather': get_current_weather,
    'get_system_time': get_system_time,
}

In [4]:
# Define prompt template
prompt = PromptTemplate(
    template=
    """
    <|begin_of_text|>
    <|start_header_id|>system<|end_header_id|>
        You are a smart Agent. 
        You are a master at understanding what a customer wants and utilize available tools only if you have to.
    <|eot_id|>
    <|start_header_id|>user<|end_header_id|>
        Conduct a comprehensive analysis of the request provided. \n
        USER REQUEST:\n\n {initial_request} \n\n
    <|eot_id|>
    <|start_header_id|>assistant<|end_header_id|>
    """,
    input_variables=["initial_request"],
)

agent_request_generator = prompt | model_with_tools

In [5]:
# List of different sentences to test
# List of cities and states for more specific location entries
cities_and_states = [
    "New York, NY", "Los Angeles, CA", "Chicago, IL", "Houston, TX", 
    "Miami, FL", "San Francisco, CA", "Seattle, WA", "Boston, MA", 
    "Austin, TX", "Denver, CO", "Philadelphia, PA", "Phoenix, AZ", 
    "Dallas, TX", "San Diego, CA", "Atlanta, GA", "Washington D.C., DC", 
    "Orlando, FL", "Nashville, TN", "Minneapolis, MN", "Las Vegas, NV"
]

specific_requests = [
    "What is the weather today in [location]?",
    "Can you tell me the forecast for tomorrow in [location]?",
    "How's the weather in [location] today?",
    "What's the temperature like right now in [location]?",
    "Tell me about the conditions for tonight in [location]?",
    "Is it going to rain tomorrow in [location]?",
    # "What time does the museum open in [location]?"
]

In [6]:
# Generate test sentences
test_sentences = []
test_suite_size = 100
for _ in range(test_suite_size):
    city_and_state = random.choice(cities_and_states)
    specific_request = random.choice(specific_requests).replace("[location]", city_and_state)
    test_sentences.append(specific_request)

# Print the generated sentences for reference
print("Generated Test Sentences:")
for sentence in test_sentences:
    print(sentence)

Generated Test Sentences:
Is it going to rain tomorrow in Philadelphia, PA?
How's the weather in Dallas, TX today?
What's the temperature like right now in Los Angeles, CA?
How's the weather in San Diego, CA today?
What is the weather today in Chicago, IL?
Can you tell me the forecast for tomorrow in Washington D.C., DC?
Tell me about the conditions for tonight in Orlando, FL?
What is the weather today in Dallas, TX?
Tell me about the conditions for tonight in Las Vegas, NV?
Tell me about the conditions for tonight in Orlando, FL?
Can you tell me the forecast for tomorrow in Nashville, TN?
Tell me about the conditions for tonight in Houston, TX?
Tell me about the conditions for tonight in Dallas, TX?
What's the temperature like right now in Atlanta, GA?
Is it going to rain tomorrow in Las Vegas, NV?
Can you tell me the forecast for tomorrow in Philadelphia, PA?
Is it going to rain tomorrow in Seattle, WA?
Is it going to rain tomorrow in Seattle, WA?
Can you tell me the forecast for tom

In [7]:
# Initialize a counter for function calls
function_call_count = 0
counter=0
# Loop through the test sentences and invoke the model with each one
for sentence in test_sentences:
    counter = counter+1
    print("----------- Loop # ",counter, "--------------")
    result = agent_request_generator.invoke({"initial_request": sentence})
    
    # Extract the last AI message from messages
    last_ai_message = None
    if isinstance(result, AIMessage):
        last_ai_message = result

    # Print information about the tool calls
    if last_ai_message and hasattr(last_ai_message, 'tool_calls'):
        print("Sentence: ", sentence)
        print("Last AI Message:", last_ai_message)
        if len(last_ai_message.tool_calls) > 0:
            # Increment the function call counter for each tool call
            function_call_count += 1
            print("Tool Name :: ", last_ai_message.tool_calls[-1]["name"])
            print("Tool Args :: ", last_ai_message.tool_calls[-1]["args"])
    else:
        print("No tool calls found for Sentence:", sentence)
    
    # Run the tool if there are any tool calls
    if hasattr(last_ai_message, 'tool_calls'):
        for tool_call in last_ai_message.tool_calls:
            tool = tool_mapping[tool_call["name"].lower()]
            tool_output = tool.invoke(tool_call["args"])
            print("Tool Output:", tool_output)
    else:
        print("No tools to run for Sentence:", sentence)
    
    # Add a newline for better readability between different sentences' results
    print("\n")
    print("Total function calls made by the AI model:", function_call_count)
    print("\n \n \n")

#print percentage of function calls vs test_suite_size
print("Success Rate :: ", function_call_count/test_suite_size*100, " percent")

----------- Loop #  1 --------------
Sentence:  Is it going to rain tomorrow in Philadelphia, PA?
Last AI Message: content='' additional_kwargs={'tool_calls': [{'id': 'call_b71d', 'function': {'arguments': '{"location": "Philadelphia, PA"}', 'name': 'get_current_weather'}, 'type': 'function'}]} response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 834, 'total_tokens': 851, 'completion_time': 0.068, 'prompt_time': 0.150275675, 'queue_time': 0.0034561130000000106, 'total_time': 0.218275675}, 'model_name': 'llama-3.1-70b-versatile', 'system_fingerprint': 'fp_b6828be2c9', 'finish_reason': 'tool_calls', 'logprobs': None} id='run-f24882ca-9683-4cf9-b5cb-0d4d54c2871e-0' tool_calls=[{'name': 'get_current_weather', 'args': {'location': 'Philadelphia, PA'}, 'id': 'call_b71d', 'type': 'tool_call'}] usage_metadata={'input_tokens': 834, 'output_tokens': 17, 'total_tokens': 851}
Tool Name ::  get_current_weather
Tool Args ::  {'location': 'Philadelphia, PA'}
Tool Output: Rainy